In [1]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [2]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.processor.utils import Utils
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [4]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [5]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()

In [6]:
pm = Portfolio.from_dict(portfolio_dictionary)
print(portfolio_dictionary)

{'ranking_metric': 'option_price', 'timeframe': 'DAY', 'position_type': 'long', 'grouping_type': 'STANDARD_DEV', 'selection_type': 'top_blacklist', 'allocation_type': 'equal', 'risk_type': 'coefficient_of_variation', 'selection_percentage': 1, 'stoploss': 1, 'num_of_groups': 1, 'max_price': 1, 'min_price': 0.01, 'max_market_cap': 10000000, 'min_market_cap': 1000000, 'rolling_window': 10, 'leverage': 1, 'date': Timestamp('2024-12-30 00:00:00'), 'pnl': 1.043856400471294, 'raw_pnl': 434876.81955935754, 'downside': 0.9996019124900113, 'coefficient_of_variation': 90.36045544037917, 'portfolio_std': 0.011552137440918965, 'sharpe_ratio': -107.88944770537388, 'tracking_error': 0.32832825736480453, 'information_ratio': -3.796059889333553}


In [7]:
## Retrieve existing stocks 
index = alp.assets().sort_values("ticker")
tickers = index["ticker"].values
tickers_per_batch = int(1000)
batchs = [tickers[i:i + tickers_per_batch] for i in range(0, len(tickers), tickers_per_batch)]
bars = []
[bars.append(alp.latest_bars_bulk(batch)) for batch in batchs]
todays_bar = pd.concat(bars)
index = index.merge(todays_bar,on="ticker",how="left")

'SPYA'
'SVA'


In [8]:
end = Utils.last_weekday(alp.clock())
start = (end - timedelta(days=pm.rolling_window*3))
print(start,end)

2025-05-03 16:00:00-04:00 2025-06-02 16:00:00-04:00


In [9]:
tickers_per_batch = int(10000/(pm.rolling_window*4)/5)
print(tickers_per_batch)

50


In [10]:
relevant_tickers = index["ticker"]
batchs = [relevant_tickers[i:i + tickers_per_batch] for i in range(0, len(relevant_tickers), tickers_per_batch)]

In [15]:
prices = []
for batch in tqdm(batchs):
    try:
        tickers_data = alp.prices_bulk(batch,start,end)
        for ticker in batch:
            try:
                price = tickers_data[tickers_data["ticker"] == ticker].copy()
                price = p.lower_column(price)
                price = p.utc_date(price)
                price.sort_values("date", inplace=True)
                price = p.additional_date_columns(price)
                price["coefficient_of_variation"] = 1
                price["dividend"] = 0
                if price.index.size > pm.rolling_window:
                    for member in Metric:
                        price = member.calculate(price,timeframe=pm.rolling_window,live=True)
                    for member in Indicator:
                        price = member.calculate(price,timeframe=pm.rolling_window,live=True)
                prices.append(price)
            except Exception as e:
                print(str(e))
    except Exception as e:
        print(str(e))

 84%|███████████████████████████████████████████████████████████████████████              | 189/226 [08:24<01:38,  2.65s/it]

'SPYA'


 85%|████████████████████████████████████████████████████████████████████████▏            | 192/226 [08:31<01:28,  2.61s/it]

'SVA'


100%|█████████████████████████████████████████████████████████████████████████████████████| 226/226 [10:00<00:00,  2.66s/it]


In [16]:
simulation

,date,open,adjclose,high,low,volume,ticker,year,month,quarter,week,day,weekday,coefficient_of_variation,dividend
0,2025-05-05 00:00:00+00:00,108.1000,108.3700,109.2500,107.4600,1385558,A,2025,5,2,19,125,0,1,0
0,2025-05-05 00:00:00+00:00,44.7300,44.5881,44.8499,44.3400,31561,DBEU,2025,5,2,19,125,0,1,0
0,2025-05-05 00:00:00+00:00,25.9700,25.5029,25.9700,25.3400,59489,DBEM,2025,5,2,19,125,0,1,0
0,2025-05-05 00:00:00+00:00,43.7100,43.7900,43.9300,43.6900,461092,DBEF,2025,5,2,19,125,0,1,0
0,2025-05-05 00:00:00+00:00,45.7800,44.7550,45.9800,44.6200,74056,VEON,2025,5,2,19,125,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,2025-06-02 00:00:00+00:00,13.7700,13.7600,13.7735,13.7000,49539,ISD,2025,6,2,23,153,0,1,0
19,2025-06-02 00:00:00+00:00,123.4900,123.8200,124.0150,121.7350,601333,SUI,2025,6,2,23,153,0,1,0
19,2025-06-02 00:00:00+00:00,0.0152,0.0152,0.0152,0.0146,900,CDIOW,2025,6,2,23,153,0,1,0
14,2025-06-02 00:00:00+00:00,27.3000,27.0100,28.0000,27.0001,2812,ISBA,2025,6,2,23,153,0,1,0


In [17]:
simulation = pd.concat(prices)
simulation.sort_values("date", inplace=True)
trades = pm.recs(simulation.copy())
trades.to_dict("records")

[{'year': 2025,
  'day': 153,
  'ticker': 'HBIO',
  'date': Timestamp('2025-06-02 00:00:00+0000', tz='UTC'),
  'open': 0.4727,
  'adjclose': 0.4054,
  'high': 0.5595,
  'low': 0.4052,
  'volume': 19565502,
  'month': 6,
  'quarter': 2,
  'week': 23,
  'weekday': 0,
  'risk': 1,
  'dividend': 0.0,
  'adjclose_test': 0.467,
  'high_test': 0.5814,
  'low_test': 0.4305,
  'volume_test': 13569171.0,
  'average_return': 0.13552928364149322,
  'standard_dev': 0.11149195287353948,
  'market_cap': 7931854.510799999,
  'simple_moving_average': 0.387,
  'drawdown': 0.6531820424272324,
  'distance': 1.9999703428668083,
  'cooked_return': 57.509932443679936,
  'next_close': nan,
  'next_open': nan,
  'price_to_return': 285.54714494301163,
  'optimal': nan,
  'adr': 0.2550220313135566,
  'sma': -0.04538727183029101,
  'ema': 0.00247708340235242,
  'bollinger_upper': 1.5046470294698544,
  'bollinger_lower': 0.4045784268695635,
  'momentum': 0.06579999999999997,
  'roc': -0.13190578158458255,
  'willi